# Bernoulli Verification Example - Temperature Loads

In [1]:
from sigmaepsilon.solid.fem.cells import B2 as Beam

# units in kN, cm and Celsius
L, d, t = 100.0, 1.2, 0.4
Ex, nu = 21000.0, 0.25
dT = 20
alpha = 1.2e-5

## Linear Solution

In [2]:
from sigmaepsilon.solid import Structure, LineMesh, PointData
from polymesh.space import StandardFrame, PointCloud
from polymesh.utils.space import index_of_closest_point
from sigmaepsilon.solid import BeamSection
from linkeddeepdict.tools import getallfromkwargs
from neumann.linalg import linspace
from neumann import repeat
import numpy as np

# section
section = BeamSection('CHS', d=d, t=t, n=16)
section.calculate_section_properties()
section_props = section.section_properties
A, Ix, Iy, Iz = getallfromkwargs(['A', 'Ix', 'Iy', 'Iz'], **section_props)

# material
G = Ex / (2 * (1 + nu))
Hooke = np.array([
    [Ex*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, Ex*Iy, 0],
    [0, 0, 0, Ex*Iz]
])

i_first, i_last = None, None


def build_structure(n: int):
    global i_first, i_last
    
    # space
    GlobalFrame = StandardFrame(dim=3)

    # mesh
    p0 = np.array([0., 0., 0.])
    p1 = np.array([L, 0., 0.])
    coords = linspace(p0, p1, n+1)
    coords = PointCloud(coords, frame=GlobalFrame).show()
    topo = np.zeros((n, 2), dtype=int)
    topo[:, 0] = np.arange(n)
    topo[:, 1] = np.arange(n) + 1
    i_first = index_of_closest_point(coords, np.array([0, 0, 0]))
    i_last = index_of_closest_point(coords, np.array([L, 0, 0]))
    
    # essential boundary conditions
    fixity = np.zeros((coords.shape[0], 6)).astype(bool)
    fixity[i_first, :] = True
    fixity[i_last, :] = True

    # natural boundary conditions
    nLoadCase = 2
    # nodal loads
    nodal_loads = np.zeros((coords.shape[0], 6, nLoadCase))
    # cell loads
    strain_loads = np.zeros((topo.shape[0], 4, nLoadCase))
    strain_loads[:, 0, 0] = dT * alpha
    strain_loads[:, 0, 1] = - dT * alpha
        
    # pointdata
    pd = PointData(coords=coords, fixity=fixity, loads=nodal_loads, 
                   frame=GlobalFrame)

    # celldata
    frames = repeat(GlobalFrame.show(), topo.shape[0])
    cd = Beam(topo=topo, frames=frames, strain_loads=strain_loads, 
              material=Hooke)

    # set up mesh and structure
    mesh = LineMesh(pd, cd, frame=GlobalFrame)
    structure = Structure(mesh=mesh)
    return structure


## Verification

In [3]:
structure = build_structure(2)
structure.linsolve()
dofsol = structure.nodal_dof_solution()
reactions = structure.reaction_forces()
forces = structure.internal_forces()

In [4]:
Ex * A * alpha * dT

4.937534717750992

In [5]:
reactions[i_first, 0, 0], reactions[i_last, 0, 0]

(4.937534715719387, -4.937534715719387)

In [6]:
reactions[i_first, 0, 1], reactions[i_last, 0, 1]

(-4.937534715719387, 4.937534715719387)

In [7]:
forces[0, 0, 0, 0], forces[-1, 0, 0, 0]

(-4.937534715719387, -4.937534715719387)

In [8]:
forces[0, 0, 0, 1], forces[-1, 0, 0, 1]

(4.937534715719387, 4.937534715719387)